In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.covidtracker.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains scripts that download data from data.gouv.fr and then process it to build many graphes.
I'm currently cleaning the code, please ask me if something is not clear enough.

The charts are exported to 'charts/images/france'.
Data is download to/imported from 'data/france'.
Requirements: please see the imports below (use pip3 to install them).

"""

"\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.covidtracker.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:\nThis file contains scripts that download data from data.gouv.fr and then process it to build many graphes.\nI'm currently cleaning the code, please ask me if something is not clear enough.\n\nThe charts are exported to 'charts/images/france'.\nData is download to/imported from 'data/france'.\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [2]:
import pandas as pd
import plotly.graph_objects as go
import france_data_management as data
from datetime import datetime
from datetime import timedelta
import plotly
import math
import os

In [3]:
df, df_confirmed, dates, df_new, df_tests, df_deconf, df_sursaud, df_incid, df_tests_viros = data.import_data()

  0%|          | 0/4 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.

 75%|███████▌  | 3/4 [00:03<00:01,  1.09s/it]

In [4]:
df_departements = df.groupby(["jour", "departmentName"]).sum().reset_index()
df_incid_departements = df_incid[df_incid["cl_age90"]==0].groupby(["jour", "departmentName"]).sum().reset_index()

departements = list(dict.fromkeys(list(df_departements['departmentName'].values))) 

dates_incid = list(dict.fromkeys(list(df_incid['jour'].values))) 
last_day_plot = (datetime.strptime(max(dates), '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d")

departements_nb = list(dict.fromkeys(list(df_tests_viros['dep'].values))) 

In [9]:
def cas_journ(departement):
        
    df_incid_dep = df_incid_departements[df_incid_departements["departmentName"] == departement]
    df_incid_dep_rolling = df_incid_dep["P"].rolling(window=7, center=True).mean()
    
    range_x, name_fig, range_y = ["2020-03-29", last_day_plot], "cas_journ_"+departement, [0, df_incid_dep["P"].max()]
    title = "<b>Cas positifs</b> au Covid19 - " + departement

    fig = go.Figure()


    fig.add_trace(go.Scatter(
        x = df_incid_dep["jour"],
        y = df_incid_dep_rolling,
        name = "Nouveaux décès hosp.",
        marker_color='rgb(8, 115, 191)',
        line_width=8,
        opacity=0.8,
        fill='tozeroy',
        fillcolor="rgba(8, 115, 191, 0.3)",
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x = [dates_incid[-4]],
        y = [df_incid_dep_rolling.values[-4]],
        name = "Nouveaux décès hosp.",
        mode="markers",
        marker_color='rgb(8, 115, 191)',
        marker_size=15,
        opacity=1,
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x = df_incid_dep["jour"],
        y = df_incid_dep["P"],
        name = "",
        mode="markers",
        marker_color='rgb(8, 115, 191)',
        line_width=3,
        opacity=0.4,
        showlegend=False
    ))

    ###

    fig.update_yaxes(zerolinecolor='Grey', range=range_y, tickfont=dict(size=18))
    fig.update_xaxes(nticks=10, ticks='inside', tickangle=0, tickfont=dict(size=18))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(
        margin=dict(
                l=50,
                r=0,
                b=50,
                t=70,
                pad=0
            ),
        legend_orientation="h",
        barmode='group',
        title={
                    'text': title,
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    titlefont = dict(
                    size=20),
        xaxis=dict(
                title='',
                tickformat='%d/%m'),

        annotations = [
                    dict(
                        x=0,
                        y=1,
                        xref='paper',
                        yref='paper',
                        text='Date : {}. Source : Santé publique France. Auteur : guillaumerozier.fr.'.format(datetime.strptime(max(dates), '%Y-%m-%d').strftime('%d %B %Y')),                    showarrow = False
                    ),
                    ]
                     )

    fig['layout']['annotations'] += (dict(
            x = dates_incid[-4], y = df_incid_dep_rolling.values[-4], # annotation point
            xref='x1', 
            yref='y1',
            text=" <b>{} {}".format('%d' % df_incid_dep_rolling.values[-4], "cas quotidiens<br></b>en moyenne du {} au {}.".format(datetime.strptime(dates_incid[-7], '%Y-%m-%d').strftime('%d'), datetime.strptime(dates_incid[-1], '%Y-%m-%d').strftime('%d %b'))),
            xshift=-2,
            yshift=10,
            xanchor="center",
            align='center',
            font=dict(
                color="rgb(8, 115, 191)",
                size=20
                ),
            bgcolor="rgba(255, 255, 255, 0.6)",
            opacity=1,
            ax=-40,
            ay=-70,
            arrowcolor="rgb(8, 115, 191)",
            arrowsize=1.5,
            arrowwidth=1,
            arrowhead=0,
            showarrow=True
        ),)

    fig.write_image("images/charts/france/departements_dashboards/{}.jpeg".format(name_fig), scale=0.9, width=750, height=500)

    print("> " + name_fig)

In [10]:
def hosp_journ(departement):   
    df_dep = df_departements[df_departements["departmentName"] == departement]
    #df_incid_reg_rolling = df_incid_reg["P"].rolling(window=7, center=True).mean()
    
    range_x, name_fig = ["2020-03-29", last_day_plot], "hosp_journ_"+departement
    title = "<b>Personnes hospitalisées</b> pour Covid19 - " + departement

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = df_dep["jour"],
        y = df_dep["hosp"],
        name = "Nouveaux décès hosp.",
        marker_color='rgb(209, 102, 21)',
        line_width=8,
        opacity=0.8,
        fill='tozeroy',
        fillcolor="rgba(209, 102, 21,0.3)",
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x = [dates[-1]],
        y = [df_dep["hosp"].values[-1]],
        name = "Nouveaux décès hosp.",
        mode="markers",
        marker_color='rgb(209, 102, 21)',
        marker_size=15,
        opacity=1,
        showlegend=False
    ))

    ###

    fig.update_yaxes(zerolinecolor='Grey', tickfont=dict(size=18))
    fig.update_xaxes(nticks=10, ticks='inside', tickangle=0, tickfont=dict(size=18))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(
        margin=dict(
                l=50,
                r=0,
                b=50,
                t=70,
                pad=0
            ),
        legend_orientation="h",
        barmode='group',
        title={
                    'text': title,
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    titlefont = dict(
                    size=20),
        xaxis=dict(
                title='',
                tickformat='%d/%m'),

        annotations = [
                    dict(
                        x=0,
                        y=1,
                        xref='paper',
                        yref='paper',
                        text='Date : {}. Source : Santé publique France. Auteur : guillaumerozier.fr.'.format(datetime.strptime(max(dates), '%Y-%m-%d').strftime('%d %B %Y')),                    showarrow = False
                    ),
                    ]
                     )

    fig['layout']['annotations'] += (dict(
            x = dates[-1], y = df_dep["hosp"].values[-1], # annotation point
            xref='x1', 
            yref='y1',
            text=" <b>{} {}".format('%d' % df_dep["hosp"].values[-1], "personnes<br>hospitalisées</b><br>le {}.".format(datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %b'))),
            xshift=-2,
            yshift=10,
            xanchor="center",
            align='center',
            font=dict(
                color="rgb(209, 102, 21)",
                size=20
                ),
            bgcolor="rgba(255, 255, 255, 0.6)",
            opacity=0.8,
            ax=-50,
            ay=-90,
            arrowcolor="rgb(209, 102, 21)",
            arrowsize=1.5,
            arrowwidth=1,
            arrowhead=0,
            showarrow=True
        ),)

    fig.write_image("images/charts/france/departements_dashboards/{}.jpeg".format(name_fig), scale=0.9, width=750, height=500)

    print("> " + name_fig)


In [11]:
def rea_journ(departement):
    df_dep = df_departements[df_departements["departmentName"] == departement]
    
    range_x, name_fig = ["2020-03-29", last_day_plot], "rea_journ_" + departement
    title = "<b>Personnes en réanimation</b> pour Covid19 - " + departement

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = dates,
        y = df_dep["rea"],
        name = "Nouveaux décès hosp.",
        marker_color='rgb(201, 4, 4)',
        line_width=8,
        opacity=0.8,
        fill='tozeroy',
        fillcolor="rgba(201, 4, 4,0.3)",
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x = [dates[-1]],
        y = [df_dep["rea"].values[-1]],
        name = "Nouveaux décès hosp.",
        mode="markers",
        marker_color='rgb(201, 4, 4)',
        marker_size=15,
        opacity=1,
        showlegend=False
    ))

    ###

    fig.update_yaxes(zerolinecolor='Grey', tickfont=dict(size=18))
    fig.update_xaxes(nticks=10, ticks='inside', tickangle=0, tickfont=dict(size=18))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(
        margin=dict(
                l=50,
                r=0,
                b=50,
                t=70,
                pad=0
            ),
        legend_orientation="h",
        barmode='group',
        title={
                    'text': title,
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    titlefont = dict(
                    size=20),
        xaxis=dict(
                title='',
                tickformat='%d/%m'),

        annotations = [
                    dict(
                        x=0,
                        y=1,
                        xref='paper',
                        yref='paper',
                        text='Date : {}. Source : Santé publique France. Auteur : guillaumerozier.fr.'.format(datetime.strptime(max(dates), '%Y-%m-%d').strftime('%d %B %Y')),                    showarrow = False
                    ),
                    ]
                     )

    fig['layout']['annotations'] += (dict(
            x = dates[-1], y = df_dep["rea"].values[-1], # annotation point
            xref='x1', 
            yref='y1',
            text=" <b>{} {}".format('%d' % df_dep["rea"].values[-1], "personnes<br>en réanimation</b><br>le {}.".format(datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %b'))),
            xshift=-2,
            yshift=10,
            xanchor="center",
            align='center',
            font=dict(
                color="rgb(201, 4, 4)",
                size=20
                ),
            bgcolor="rgba(255, 255, 255, 0.6)",
            opacity=0.8,
            ax=-50,
            ay=-90,
            arrowcolor="rgb(201, 4, 4)",
            arrowsize=1.5,
            arrowwidth=1,
            arrowhead=0,
            showarrow=True
        ),)

    fig.write_image("images/charts/france/departements_dashboards/{}.jpeg".format(name_fig), scale=0.9, width=750, height=500)

    print("> " + name_fig)


In [12]:
def dc_journ(departement): 
    df_dep = df_departements[df_departements["departmentName"] == departement]
    dc_new_rolling = df_dep["dc_new"].rolling(window=7).mean()
    
    range_x, name_fig, range_y = ["2020-03-29", last_day_plot], "dc_journ_"+departement, [0, df_dep["dc_new"].max()]
    title = "<b>Décès hospitaliers quotidiens</b> du Covid19 - " + departement

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x = df_dep["jour"],
        y = dc_new_rolling,
        name = "Nouveaux décès hosp.",
        marker_color='black',
        line_width=8,
        opacity=0.8,
        fill='tozeroy',
        fillcolor="rgba(0,0,0,0.3)",
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x = [dates[-1]],
        y = [dc_new_rolling.values[-1]],
        name = "Nouveaux décès hosp.",
        mode="markers",
        marker_color='black',
        marker_size=15,
        opacity=1,
        showlegend=False
    ))

    #
    fig.add_trace(go.Scatter(
        x = df_dep["jour"],
        y = df_dep["dc_new"],
        name = "Nouveaux décès hosp.",
        mode="markers",
        marker_color='black',
        line_width=3,
        opacity=0.4,
        showlegend=False
    ))

    ###

    fig.update_yaxes(zerolinecolor='Grey', range=range_y, tickfont=dict(size=18))
    fig.update_xaxes(nticks=10, ticks='inside', tickangle=0, tickfont=dict(size=18))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(
        margin=dict(
                l=50,
                r=0,
                b=50,
                t=70,
                pad=0
            ),
        legend_orientation="h",
        barmode='group',
        title={
                    'text': title,
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                    titlefont = dict(
                    size=20),
        xaxis=dict(
                title='',
                tickformat='%d/%m'),

        annotations = [
                    dict(
                        x=0,
                        y=1,
                        xref='paper',
                        yref='paper',
                        text='Date : {}. Source : Santé publique France. Auteur : guillaumerozier.fr.'.format(datetime.strptime(max(dates), '%Y-%m-%d').strftime('%d %B %Y')),                    showarrow = False
                    ),
                    ]
                     )

    fig['layout']['annotations'] += (dict(
            x = dates[-1], y = dc_new_rolling.values[-1], # annotation point
            xref='x1', 
            yref='y1',
            text=" <b>{} {}".format('%d' % math.trunc(round(dc_new_rolling.values[-1], 2)), "décès quotidiens</b><br>en moyenne<br>du {} au {}.".format(datetime.strptime(dates[-7], '%Y-%m-%d').strftime('%d'), datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %b'))),
            xshift=-2,
            yshift=10,
            xanchor="center",
            align='center',
            font=dict(
                color="black",
                size=20
                ),
            bgcolor="rgba(255, 255, 255, 0.6)",
            opacity=0.8,
            ax=-50,
            ay=-90,
            arrowcolor="black",
            arrowsize=1.5,
            arrowwidth=1,
            arrowhead=0,
            showarrow=True
        ),)

    fig.write_image("images/charts/france/departements_dashboards/{}.jpeg".format(name_fig), scale=0.9, width=750, height=500)

    print("> " + name_fig)


In [13]:
import cv2

for dep in departements:
    cas_journ(dep)
    hosp_journ(dep)
    rea_journ(dep)
    dc_journ(dep)
    
    
    im1 = cv2.imread('images/charts/france/departements_dashboards/cas_journ_{}.jpeg'.format(dep))
    im2 = cv2.imread('images/charts/france/departements_dashboards/hosp_journ_{}.jpeg'.format(dep))
    im3 = cv2.imread('images/charts/france/departements_dashboards/rea_journ_{}.jpeg'.format(dep))
    im4 = cv2.imread('images/charts/france/departements_dashboards/dc_journ_{}.jpeg'.format(dep))

    im_haut = cv2.hconcat([im1, im2])
    #cv2.imwrite('images/charts/france/tests_combinaison.jpeg', im_h)
    im_bas = cv2.hconcat([im3, im4])

    im_totale = cv2.vconcat([im_haut, im_bas])
    cv2.imwrite('images/charts/france/departements_dashboards/dashboard_jour_{}.jpeg'.format(dep), im_totale)
    
    os.remove('images/charts/france/departements_dashboards/cas_journ_{}.jpeg'.format(dep))
    os.remove('images/charts/france/departements_dashboards/hosp_journ_{}.jpeg'.format(dep))
    os.remove('images/charts/france/departements_dashboards/rea_journ_{}.jpeg'.format(dep))
    os.remove('images/charts/france/departements_dashboards/dc_journ_{}.jpeg'.format(dep))

> cas_journ_Ain
> hosp_journ_Ain
> rea_journ_Ain
> dc_journ_Ain
> cas_journ_Aisne
> hosp_journ_Aisne
> rea_journ_Aisne
> dc_journ_Aisne
> cas_journ_Allier
> hosp_journ_Allier
> rea_journ_Allier
> dc_journ_Allier
> cas_journ_Alpes-Maritimes
> hosp_journ_Alpes-Maritimes
> rea_journ_Alpes-Maritimes
> dc_journ_Alpes-Maritimes
> cas_journ_Alpes-de-Haute-Provence
> hosp_journ_Alpes-de-Haute-Provence
> rea_journ_Alpes-de-Haute-Provence
> dc_journ_Alpes-de-Haute-Provence
> cas_journ_Ardennes
> hosp_journ_Ardennes
> rea_journ_Ardennes
> dc_journ_Ardennes
> cas_journ_Ardèche
> hosp_journ_Ardèche
> rea_journ_Ardèche
> dc_journ_Ardèche
> cas_journ_Ariège
> hosp_journ_Ariège
> rea_journ_Ariège
> dc_journ_Ariège
> cas_journ_Aube
> hosp_journ_Aube
> rea_journ_Aube
> dc_journ_Aube
> cas_journ_Aude
> hosp_journ_Aude
> rea_journ_Aude
> dc_journ_Aude
> cas_journ_Aveyron
> hosp_journ_Aveyron
> rea_journ_Aveyron
> dc_journ_Aveyron
> cas_journ_Bas-Rhin
> hosp_journ_Bas-Rhin
> rea_journ_Bas-Rhin
> dc_journ_B

> hosp_journ_Val-d'oise
> rea_journ_Val-d'oise
> dc_journ_Val-d'oise
> cas_journ_Val-de-Marne
> hosp_journ_Val-de-Marne
> rea_journ_Val-de-Marne
> dc_journ_Val-de-Marne
> cas_journ_Var
> hosp_journ_Var
> rea_journ_Var
> dc_journ_Var
> cas_journ_Vaucluse
> hosp_journ_Vaucluse
> rea_journ_Vaucluse
> dc_journ_Vaucluse
> cas_journ_Vendée
> hosp_journ_Vendée
> rea_journ_Vendée
> dc_journ_Vendée
> cas_journ_Vienne
> hosp_journ_Vienne
> rea_journ_Vienne
> dc_journ_Vienne
> cas_journ_Vosges
> hosp_journ_Vosges
> rea_journ_Vosges
> dc_journ_Vosges
> cas_journ_Yonne
> hosp_journ_Yonne
> rea_journ_Yonne
> dc_journ_Yonne
> cas_journ_Yvelines
> hosp_journ_Yvelines
> rea_journ_Yvelines
> dc_journ_Yvelines


In [38]:
for idx,dep in enumerate(departements):
    numero_dep = df[df["departmentName"] == dep]["dep"].values[-1]
    
    heading = "<!-- wp:heading --><h2 id=\"{}\">{}</h2><!-- /wp:heading -->\n".format(dep, dep + " (" + numero_dep + ")")
    string = "<p align=\"center\"> <a href=\"https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_{}.jpeg\" target=\"_blank\" rel=\"noopener noreferrer\"><img src=\"https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_{}.jpeg\" width=\"75%\"> </a></p>\n".format(dep, dep)
    string2 = "<p align=\"center\"> <a href=\"https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_{}.jpeg\" target=\"_blank\" rel=\"noopener noreferrer\"><img src=\"https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_{}.jpeg\" width=\"60%\"> </a></p>\n".format(numero_dep, numero_dep)
    space = "<!-- wp:spacer {\"height\":50} --><div style=\"height:50px\" aria-hidden=\"true\" class=\"wp-block-spacer\"></div><!-- /wp:spacer -->"
    retourmenu="<a href=\"#Menu\">Retour au menu</a>"
    print(space+retourmenu+heading+string+string2)


<!-- wp:spacer {"height":50} --><div style="height:50px" aria-hidden="true" class="wp-block-spacer"></div><!-- /wp:spacer --><a href="#Menu">Retour au menu</a><!-- wp:heading --><h2 id="Ain">Ain (01)</h2><!-- /wp:heading -->
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Ain.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Ain.jpeg" width="75%"> </a></p>
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_01.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_01.jpeg" width="60%"> </a></p>

<!-- wp:spacer {"height":50} --><div style="height:5

<!-- wp:spacer {"height":50} --><div style="height:50px" aria-hidden="true" class="wp-block-spacer"></div><!-- /wp:spacer --><a href="#Menu">Retour au menu</a><!-- wp:heading --><h2 id="Essonne">Essonne (91)</h2><!-- /wp:heading -->
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Essonne.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Essonne.jpeg" width="75%"> </a></p>
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_91.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_91.jpeg" width="60%"> </a></p>

<!-- wp:spacer {"height":50} --><div

<!-- wp:spacer {"height":50} --><div style="height:50px" aria-hidden="true" class="wp-block-spacer"></div><!-- /wp:spacer --><a href="#Menu">Retour au menu</a><!-- wp:heading --><h2 id="Hautes-Alpes">Hautes-Alpes (05)</h2><!-- /wp:heading -->
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Hautes-Alpes.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Hautes-Alpes.jpeg" width="75%"> </a></p>
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_05.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_05.jpeg" width="60%"> </a></p>

<!-- wp:spacer {

<!-- wp:spacer {"height":50} --><div style="height:50px" aria-hidden="true" class="wp-block-spacer"></div><!-- /wp:spacer --><a href="#Menu">Retour au menu</a><!-- wp:heading --><h2 id="Pas-de-Calais">Pas-de-Calais (62)</h2><!-- /wp:heading -->
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Pas-de-Calais.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/departements_dashboards/dashboard_jour_Pas-de-Calais.jpeg" width="75%"> </a></p>
<p align="center"> <a href="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_62.jpeg" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/rozierguillaume/covid-19/master/images/charts/france/heatmaps_deps/heatmap_taux_62.jpeg" width="60%"> </a></p>

<!-- wp:spac

In [49]:
#print("<!-- wp:buttons --><div class=\"wp-block-buttons\">\n")
output = ""
for dep in departements:
    numero_dep = df[df["departmentName"] == dep]["dep"].values[-1]
    output+= "<a href=\"#{}\">{} ({})</a> • ".format(dep, dep, numero_dep)
    #print("""<!-- wp:button {"className":"is-style-outline"} -->
    #<div class="wp-block-button is-style-outline">""")
    #print("<a class=\"wp-block-button__link\" href=\"#{}\">".format(dep))
    #print("{}</a></div><!-- /wp:button --></div>\n".format(dep + " (" + numero_dep + ")"))
print(output[:-2])
    
    
#print("<!-- /wp:buttons -->")

<a href="#Ain">Ain (01)</a> • <a href="#Aisne">Aisne (02)</a> • <a href="#Allier">Allier (03)</a> • <a href="#Alpes-Maritimes">Alpes-Maritimes (06)</a> • <a href="#Alpes-de-Haute-Provence">Alpes-de-Haute-Provence (04)</a> • <a href="#Ardennes">Ardennes (08)</a> • <a href="#Ardèche">Ardèche (07)</a> • <a href="#Ariège">Ariège (09)</a> • <a href="#Aube">Aube (10)</a> • <a href="#Aude">Aude (11)</a> • <a href="#Aveyron">Aveyron (12)</a> • <a href="#Bas-Rhin">Bas-Rhin (67)</a> • <a href="#Bouches-du-Rhône">Bouches-du-Rhône (13)</a> • <a href="#Calvados">Calvados (14)</a> • <a href="#Cantal">Cantal (15)</a> • <a href="#Charente">Charente (16)</a> • <a href="#Charente-Maritime">Charente-Maritime (17)</a> • <a href="#Cher">Cher (18)</a> • <a href="#Corrèze">Corrèze (19)</a> • <a href="#Corse-du-Sud">Corse-du-Sud (2A)</a> • <a href="#Creuse">Creuse (23)</a> • <a href="#Côte-d'or">Côte-d'or (21)</a> • <a href="#Côtes-d'armor">Côtes-d'armor (22)</a> • <a href="#Deux-Sèvres">Deux-Sèvres (79)</a> 

In [51]:
df_incid[df_incid["dep"] == "974"]

,dep,jour,pop,P,cl_age90,departmentCode,departmentName,regionCode,regionName,T,incidence,incidence_color
174240,974,2020-05-13,121691.0,0,9,974,La Réunion,4.0,La Réunion,355,NaN,Risque Faible
174241,974,2020-05-13,134452.0,0,19,974,La Réunion,4.0,La Réunion,355,NaN,Risque Faible
174242,974,2020-05-13,98215.0,1,29,974,La Réunion,4.0,La Réunion,355,NaN,Risque Faible
174243,974,2020-05-13,104568.0,0,39,974,La Réunion,4.0,La Réunion,355,NaN,Risque Faible
174244,974,2020-05-13,113615.0,0,49,974,La Réunion,4.0,La Réunion,355,NaN,Risque Faible
...,...,...,...,...,...,...,...,...,...,...,...,...
175995,974,2020-10-19,89851.0,9,69,974,La Réunion,4.0,La Réunion,1721,36.727471,Risque Faible
175996,974,2020-10-19,46693.0,2,79,974,La Réunion,4.0,La Réunion,1721,27.841432,Risque Faible
175997,974,2020-10-19,20815.0,0,89,974,La Réunion,4.0,La Réunion,1721,19.216911,Risque Faible
175998,974,2020-10-19,4462.0,0,90,974,La Réunion,4.0,La Réunion,1721,44.822949,Risque Faible
